In [6]:
#Importar librerías
import pandas as pd
import os
import zipfile
import shutil
import requests
import zipfile
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import PercentFormatter
import geopandas as gpd
import folium
from branca.element import Template, MacroElement
from scipy.stats import randint

In [12]:
#Directorio de trabajo
try:
    os.chdir('C:/users/claud/Documents/GitHub/datavivi')
except:
    os.chdir('C:/users/claudio.pacheco/Documents/Github/datavivi')

os.getcwd()

'C:\\users\\claud\\Documents\\GitHub\\datavivi'

In [13]:
#Crear carpeta para descargar los datos
if not os.path.exists('datos'):
    os.makedirs('datos')

In [14]:
#Función para descargar y extraer archivos
def download_and_extract(url, filename, check_file_path, extract_path):
    # Checar si el archivo ya existe
    if os.path.isfile(check_file_path):
        print(f"El archivo {filename} existe, no se descargará.")
    else:
        print(f"El archivo {filename} no existe. Descargando.")
        response = requests.get(url)
        with open(filename, "wb") as code:
            code.write(response.content)
        if zipfile.is_zipfile(filename):
            # extraer zip
            with zipfile.ZipFile(filename, "r") as zip_ref:
                zip_ref.extractall(extract_path)
        
        # Eliminar zip
            os.remove(filename)
        else:
            print("El archivo descargado no es un zip. Proceso terminado.")

In [27]:
#Url de los archivos
url_basica="https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_"

In [28]:
#Ciclo para descargar y extraer los archivos
for entidad in range(1,33):
    if entidad<10:
        url=url_basica+"0"+str(entidad)+"_2020_csv.zip"
        filename="datos/conjunto_de_datos_ageb_urbana_"+"0"+str(entidad)+"_cpv2020.zip"
        check_file_path="datos/conjunto_de_datos_ageb_urbana_"+"0"+str(entidad)+"_cpv2020.csv"
        extract_path="datos"
        download_and_extract(url, filename, check_file_path, extract_path)
    else:
        url=url_basica+str(entidad)+"_2020_csv.zip"
        filename="datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.zip"
        check_file_path="datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv"
        extract_path="datos"
        download_and_extract(url, filename, check_file_path, extract_path)

El archivo datos/conjunto_de_datos_ageb_urbana_01_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_02_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_03_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_04_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_05_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_06_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_07_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_08_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_09_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_10_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_11_cpv2020.zip no existe. Descargando.
El archivo datos/conjunto_de_datos_ageb_urbana_12_cpv2

In [30]:
#Pegar todos los archivos csv en una sola data frame
df=pd.DataFrame()
for entidad in range(1,33):
    if entidad<10:
        df_temp=pd.read_csv("datos/RESAGEBURB_"+"0"+str(entidad)+"CSV20.csv",encoding='utf-8', low_memory=False)
    else:
        df_temp=pd.read_csv("datos/RESAGEBURB_"+str(entidad)+"CSV20.csv",encoding='utf-8', low_memory=False)
    df=pd.concat([df,df_temp],axis=0)
    print("Se ha agregado la entidad "+str(entidad)," a la dataframe")

#Imprimir el número de registros con separadores de miles
print("Se concluyó la concatenación. El conjunto de datos cuenta con "+str(df.shape[1])+" columnas y "+str("{:,}".format(df.shape[0]))+" registros")
df.columns=df.columns.str.lower() 
df

Se ha agregado la entidad 1  a la dataframe
Se ha agregado la entidad 2  a la dataframe
Se ha agregado la entidad 3  a la dataframe
Se ha agregado la entidad 4  a la dataframe
Se ha agregado la entidad 5  a la dataframe
Se ha agregado la entidad 6  a la dataframe
Se ha agregado la entidad 7  a la dataframe
Se ha agregado la entidad 8  a la dataframe
Se ha agregado la entidad 9  a la dataframe
Se ha agregado la entidad 10  a la dataframe
Se ha agregado la entidad 11  a la dataframe
Se ha agregado la entidad 12  a la dataframe
Se ha agregado la entidad 13  a la dataframe
Se ha agregado la entidad 14  a la dataframe
Se ha agregado la entidad 15  a la dataframe
Se ha agregado la entidad 16  a la dataframe
Se ha agregado la entidad 17  a la dataframe
Se ha agregado la entidad 18  a la dataframe
Se ha agregado la entidad 19  a la dataframe
Se ha agregado la entidad 20  a la dataframe
Se ha agregado la entidad 21  a la dataframe
Se ha agregado la entidad 22  a la dataframe
Se ha agregado la e

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,pobtot,pobfem,...,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
0,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la entidad,0000,0,1425607,728924,...,147818,359895,236003,174089,98724,70126,6021,15323,128996,1711
1,1,Aguascalientes,1,Aguascalientes,0,Total del municipio,0000,0,948990,486917,...,116647,251719,178619,130290,80951,56131,3299,7293,74227,731
2,1,Aguascalientes,1,Aguascalientes,1,Total de la localidad urbana,0000,0,863893,444725,...,112002,232793,169675,123670,77719,53589,2995,5984,63661,595
3,1,Aguascalientes,1,Aguascalientes,1,Total AGEB urbana,0017,0,2237,1137,...,11,625,189,352,46,74,15,23,391,*
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,170,87,...,*,53,13,33,5,5,0,*,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,2,*,...,*,*,*,*,*,*,*,*,*,*
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,0,0,...,0,0,0,0,0,0,0,0,0,0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0,...,0,0,0,0,0,0,0,0,0,0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,2,*,...,*,*,*,*,*,*,*,*,*,*
